# 21.4.1. Декорирование функций с аргументами. Знакомство с фикстурами.
*Фикстуры* — это функции, выполняемые pytest до или после тестовых функций. Код в фикстуре может делать все, что вам необходимо: подготавливать данные для теста, настраивать логирование приложения и т.д. Иными словами, фикстуры обеспечивают механизм отделения непосредственно кода тестов от «подготовки к» и «очистке после» исполнения тестов.
простой пример фикстуры, которая возвращает число:

In [ ]:
import pytest

@pytest.fixture()
def some_data():
    return 42

def test_some_data(some_data):
    assert some_data == 42

Декоратор @pytest.fixture() используется, чтобы сообщить pytest, что функция является фикстурой. Если мы включаем имя фикстуры в список параметров тестовой функции, то pytest запустит сначала саму фикстуру, а потом передаст результат её выполнения в тестовую функцию. В данном случае фикстура возвращает данные, но в фикстурах может выполняться и любая другая работа.

Тест test_some_data() имеет в качестве параметра имя фикстуры some_data, pytest определит это и найдёт фикстуру с таким названием. Наименование значимо в pytest, pytest будет искать в модуле теста фикстуру с таким именем. В случае, если такой фикстуры найдено не будет, поиск продолжится в файле conftest.py.

## Организация setup и teardown в фикстурах
Итак, внимательно посмотрев на код, мы заметили, что в каждом тестовом методе происходит одна и та же операция — получение ключа API для последующего запроса. Давайте на примере одного теста разберем, как нам использовать эту часть перед тем, как выполнится тест.

Фикстура setup. Переделаем нашу функцию инициализации класса следующим образом:

In [ ]:
def get_key():
    # переменные email и password нужно заменить своими учетными данными
    response = requests.post(url='https://petfriends.skillfactory.ru/login',
                             data={"email": email, "pass": password})
    assert response.status_code == 200, 'Запрос выполнен неуспешно'
    assert 'Cookie' in response.request.headers, 'В запросе не передан ключ авторизации'
    return response.request.headers.get('Cookie')

def test_getAllPets(get_key):
    response = requests.get(url='https://petfriends.skillfactory.ru/api/pets',
                            headers={"Cookie": get_key})
    assert response.status_code == 200, 'Запрос выполнен неуспешно'
    assert len(response.json().get('pets')) > 0, 'Количество питомцев не соответствует ожиданиям'

Логика работы фикстуры такова:

Отправляем запрос логина;
Проверяем, что код ответа от сервиса равен 200;
Проверяем, что в хедерах есть Cookie;
Возвращаем в тест Cookie.
Благодаря двум проверкам в фикстуре, если эта часть выполнится с ошибкой, то тесты закончатся сразу же на этом этапе.

Теперь давайте разберёмся с teardown. Для этого напишем фикстуру, которая будет считать, сколько времени длился тест.

In [ ]:
@pytest.fixture(autouse=True)
def time_delta():
    start_time = datetime.now()
    yield
    end_time = datetime.now()
    print (f"\nТест шел: {end_time - start_time}")

Каждый раз передавать в параметры функции-теста имя фикстуры не очень удобно. Поэтому для того чтобы фикстура вызывалась автоматически, мы можем указать в её параметрах autouse=True.

Теперь разберёмся, как же организован teardown механизм. Для этого в pytest есть специальное слово, которое позволяет посреди фикстуры вызвать выполнение тестовой функции и продолжить после того, как тестовая функция закончит делать свои операции. Слово это — yield. Достаточно написать его перед вызовом блока teardown фикстуры, и всё готово.

Логика работы фикстуры такова:

Получаем текущее время;
Переходим к выполнению теста;
Получаем текущее время еще раз;
Выводим на экран разницу между первой и второй временными отсечками.